In [ ]:
# Importação das bibliotecas:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

# Importação dos dados:
base = pd.read_csv('dataset.csv')

In [ ]:
# Tratamento dos valores:

base.loc[base['Mês'] == 'Jan', 'Mês'] = 1
base.loc[base['Mês'] == 'Fev', 'Mês'] = 2
base.loc[base['Mês'] == 'Mar', 'Mês'] = 3
base.loc[base['Mês'] == 'Abr', 'Mês'] = 4
base.loc[base['Mês'] == 'Mai', 'Mês'] = 5
base.loc[base['Mês'] == 'Jun', 'Mês'] = 6
base.loc[base['Mês'] == 'Jul', 'Mês'] = 7
base.loc[base['Mês'] == 'Ago', 'Mês'] = 8
base.loc[base['Mês'] == 'Set', 'Mês'] = 9
base.loc[base['Mês'] == 'Out', 'Mês'] = 10
base.loc[base['Mês'] == 'Nov', 'Mês'] = 11
base.loc[base['Mês'] == 'Dez', 'Mês'] = 12

base['Mês'] = base['Mês'].astype(int)
base['Dia'] = base['Dia'].astype(int)

base['Chamadas Realizadas']=base['Chamadas Realizadas'].astype(int)

base.loc[base['Status de Pagamento'] == 'Pago', 'Status de Pagamento'] = 1
base.loc[base['Status de Pagamento'] == 'Não pago', 'Status de Pagamento'] = 0 
base['Status de Pagamento'] = base['Status de Pagamento'].astype(int)

base['Valor Pago'] = base['Valor Pago'].str.lstrip('R$ ')
base['Valor Pago'] = base['Valor Pago'].astype(int)


In [ ]:
base2 = base.groupby('Equipe')['Valor Pago'].sum().reset_index()
fig1 = go.Figure(go.Bar(x=base2['Valor Pago'], y=base2['Equipe'], orientation='h', 
                        textposition='auto', text=base2['Valor Pago'], 
                        insidetextfont=dict(family='Times', size=12)))

fig1.show()

In [ ]:
base3 = base.groupby('Dia')['Chamadas Realizadas'].sum().reset_index()

fig2 = go.Figure(go.Scatter(x=base3['Dia'], y=base3['Chamadas Realizadas'], mode='lines', fill='tonexty'))
fig2.add_annotation(text='Chamadas realizadas por dia',
                    xref='paper', yref='paper',
                    font=dict(size=12, color='gray'),
                    align='center', bgcolor='rgba(0, 0, 0, 0.8)',
                    x=0.05, y=0.85, showarrow=False)
fig2.add_annotation(text=f'Média: {round(base3["Chamadas Realizadas"].mean(), 2)}',
                    xref='paper', yref='paper',
                    font=dict(size=12, color='gray'),
                    align='center', bgcolor='rgba(0, 0, 0, 0.8)',
                    x= 0.05, y=0.75, showarrow=False)
fig2.show()



In [ ]:
base4 = base.groupby('Mês')['Chamadas Realizadas'].sum().reset_index()

fig3 = go.Figure(go.Scatter(x=base4['Mês'], y=base4['Chamadas Realizadas'], mode='lines', fill='tonexty'))

fig3.add_annotation(text='Média de chamadas realizadas por mês',
                    xref='paper', yref='paper',
                    font=dict(size=12, color='gray'),
                    align='center', bgcolor='rgba(0, 0, 0, 0.8)',
                    x=0.05, y=0.85, showarrow=False)
fig3.add_annotation(text=f"Média: {round(base4['Chamadas Realizadas'].mean(), 2)}",
                    xref='paper', yref='paper',
                    font=dict(size=12, color='gray'),
                    align='center', bgcolor='rgba(0, 0, 0, 0.8)',
                    x=0.05, y=0.75, showarrow=False)

fig3.show()

In [ ]:
base5 = base.groupby(['Equipe', 'Consultor'])['Valor Pago'].sum()
base5 = base5.sort_values(ascending=False)
base5 = base5.groupby('Equipe').head(1).reset_index()

fig4 = go.Figure(go.Bar(x=base5['Consultor'], y=base5['Valor Pago'], textposition='auto'))
fig4.show()

In [ ]:
base6 = base.groupby(['Meio de Propaganda', 'Mês'])['Valor Pago'].sum().reset_index()

fig5 = px.line(base6, y="Valor Pago", x="Mês", color="Meio de Propaganda")
fig5.show()

In [ ]:
base7 = base.groupby('Meio de Propaganda')['Valor Pago'].sum().reset_index()
fig6 = go.Figure(go.Pie(labels=base7['Meio de Propaganda'], values=base7['Valor Pago'], hole=0.7))
fig6.show()

In [ ]:
# Ganhos por mês separados por equipe:

base8 = base.groupby(['Mês', 'Equipe'])['Valor Pago'].sum().reset_index()
base8_group = base8.groupby('Mês')['Valor Pago'].sum().reset_index()


fig7 = px.line(base8, y="Valor Pago", x="Mês", color="Equipe")
fig7.add_trace(go.Scatter(y=base8_group["Valor Pago"], x=base8_group["Mês"], mode='lines+markers', fill='tonexty', fillcolor= 'rgba(0, 0, 0, 0.1)', name='Total de Vendas'))
fig7.show()

In [ ]:
# Chamadas pagas e não pagas:

base9 = base.groupby('Status de Pagamento')['Chamadas Realizadas'].sum()
fig8 = go.Figure()
fig8.add_trace(go.Pie(labels=['Não Pago', 'Pago'], values=base9, hole=0.6))
fig8.show()

In [ ]:
# Indicadores melhores consultores e suas respectivas equipes:

base10 = base.groupby(['Consultor', 'Equipe'])['Valor Pago'].sum()
base10.sort_values(ascending=False, inplace=True)
base10 = base10.reset_index()

fig9=go.Figure(go.Indicator(mode='number+delta', 
                            title={"text": f"<span style='font size:150%'>{base10['Consultor'].iloc[0]} - Melhor consultor</span><br><span style='font-size:70%'>Em vendas - Em relaçao a média"},
                            value=base10['Valor Pago'].iloc[0],
                            number={'prefix': "R$"},
                            delta={'relative': True, 'valueformat':'.1%', 'reference':base10['Valor Pago'].mean()}))
fig9.show()

In [ ]:
nova_base2=base.groupby('Equipe')['Valor Pago'].sum()
nova_base2.sort_values(ascending=False, inplace=True)
nova_base2=nova_base2.reset_index()

fig10=go.Figure(go.Indicator(mode='number+delta', 
                            title={"text": f"<span style='font size:150%'>{nova_base2['Equipe'].iloc[0]} - Melhor Equipe</span><br><span style='font-size:70%'>Em vendas - Em relaçao a média"},
                            value=nova_base2['Valor Pago'].iloc[0],
                            number={'prefix': "R$"},
                            delta={'relative': True, 'valueformat':'.1%', 'reference':nova_base2['Valor Pago'].mean()}))
fig10.show()

In [ ]:
# Indicador de ganhos totais:

fig11=go.Figure(go.Indicator(mode='number', 
                            title={"text": f"<span style='font size:150%'>Ganho Total</span><br><span style='font-size:70%'>Em vendas - Em relaçao a média"},
                            value=base['Valor Pago'].sum(),
                            number={'prefix': "R$"}))
fig11.show()

In [ ]:
# Indicador de total de chamadas:

fig12=go.Figure(go.Indicator(mode='number', 
                            title={"text": f"<span style='font size:150%'>Total de Chamadas</span>"},
                            value=len(base[base['Chamadas Realizadas']==1])))
fig12.show()

In [ ]:
base11 = base.groupby(['Equipe', 'Consultor'])['Valor Pago'].sum()
base11 = base11.sort_values(ascending=False)
base11 = base11.reset_index()
base11 = base11.groupby('Equipe').head(1).reset_index()


fig13 = go.Figure(go.Pie(labels=base11['Equipe'] + ' - ' + base11['Consultor'], values=base11['Valor Pago'], hole=0.6))
fig13.show()

In [ ]:

fig14 = go.Figure(go.Bar(x=base11['Consultor'], y=base11['Valor Pago'], textposition='auto', text=base11['Valor Pago']))
fig14.show()